# GPU 확인

In [1]:
import torch

assert torch.cuda.is_available()

# Install text-generation web ui

In [2]:
%cd /content

/content


In [3]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python

In [4]:
!git clone https://github.com/oobabooga/text-generation-webui.git

fatal: destination path 'text-generation-webui' already exists and is not an empty directory.


In [5]:
!pip install -r ./text-generation-webui/requirements.txt

Ignoring llama-cpp-python: markers 'platform_system == "Linux" and platform_machine == "x86_64" and python_version == "3.11"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 15.0 MB/s eta 0:00:00
Ignoring llama-cpp-python: markers 'platform_system == "Windows" and python_version == "3.11"' don't match your environment
Ignoring llama-cpp-python: markers 'platform_system == "Windows" and python_version == "3.10"' don't match your environment
Ignoring llama-cpp-python-cuda: markers 'platform_system == "Windows" and python_version == "3.11"' don't match your environment
Ignoring llama-cpp-python-cuda: markers 'platform_system == "Windows" and python_version == "3.10"' don't match your environment
Ignoring llama-cpp-python-cuda: markers 'platform_system == "Linux" and platform_machine == "x86_64" and python_version == "3.11"' don't match your environment
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 MB 9.9 MB/s eta 0:00:00
Ignoring llama-cpp-p

# load Some file to colab local drive

In [3]:
# configure for your setting

repo_name = "yanolja/Bookworm-10.7B-v0.4-DPO"
backup_path = "/content/drive/MyDrive/자료"
qtype = "q8_0"


In [4]:
# Google Drive Mount
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
import os
from pathlib import Path
web_ui_path = Path("/content/text-generation-webui/")
%cd {str(web_ui_path)}
model_file_name = repo_name.split("/")[-1]+"_"+qtype+".bin"
model_name = repo_name.split('/')[-1]
model_file_path = Path(backup_path) / model_file_name
dest_model_path = web_ui_path / "models" / model_file_name
overwrite = True
# if there is no model file in google drive, download, quantize, copy and paste.
if not os.path.exists(model_file_path) or overwrite:
  !git clone https://github.com/ggerganov/llama.cpp.git
  %cd llama.cpp
  !git pull && make clean && LLAMA_CUDA=1 make
  !pip install -r requirements.txt
  %cd models
  !git clone https://huggingface.co/{repo_name}.git
  fp16 = f"{model_name}.gguf.fp16.bin"
  %cd ..
  !python convert.py models/{model_name} --outtype f16 --outfile models/{fp16} # convert to fp16
  !rm -rf models/{model_name}
  !./quantize models/{fp16} {dest_model_path} {qtype} # convert to q8
  !cp {dest_model_path} {model_file_path}
  !rm models/{fp16}

/content/text-generation-webui
fatal: destination path 'llama.cpp' already exists and is not an empty directory.
/content/text-generation-webui/llama.cpp
Already up to date.
I ccache not found. Consider installing it for faster compilation.
I llama.cpp build info: 
I UNAME_S:   Linux
I UNAME_P:   x86_64
I UNAME_M:   x86_64
I CFLAGS:    -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG  -std=c11   -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wshadow -Wstrict-prototypes -Wpointer-arith -Wmissing-prototypes -Werror=implicit-int -Werror=implicit-function-declaration -pthread -march=native -mtune=native -Wdouble-promotion 
I CXXFLAGS:  -std=c++11 -fPIC -O3 -Wall -Wextra -Wpedantic -Wcast-qual -Wno-unused-function -Wmissing-declarations -Wmissing-noreturn -pthread  -march=native -mtune=native -Wno-array-bounds -Wno-format-truncation -Wextra-semi -I. -Icommon -D_XOPEN_SOURCE=600 -D_GNU_SOURCE -DNDEBUG 
I NVCCFLAGS: -std=c++11 -O3 
I LDFLAGS:    
I CC:        cc (

In [15]:
assert os.path.exists(model_file_path), model_file_path
if not os.path.exists(dest_model_path):
  !cp {model_file_path} {dest_model_path}
  print("model file has been loaded")
else:
  print("model file was loaded already")
%cd {web_ui_path}

model file was loaded already
/content/text-generation-webui


# LLM Local Model Configuration

https://github.com/oobabooga/text-generation-webui/blob/main/settings-template.yaml

In [17]:
custom_config = {
    "max_new_tokens" : 1024,
    "truncation_length" : 5120,
    "instruction_template" : "LLaMA",
}

In [18]:
import yaml
from pprint import pprint

config_template_file_name = "settings-template.yaml"
config_template_path = web_ui_path / config_template_file_name

with open(config_template_path, 'r') as f:
  config  = yaml.safe_load(f)

config = {**config, **custom_config}
pprint(config)

{'add_bos_token': True,
 'auto_max_new_tokens': False,
 'autoload_model': False,
 'ban_eos_token': False,
 'character': 'Assistant',
 'chat-instruct_command': 'Continue the chat dialogue below. Write a single '
                          'reply for the character "<|character|>".\n'
                          '\n'
                          '<|prompt|>',
 'chat_style': 'cai-chat',
 'chat_template_str': '{%- for message in messages %}\n'
                      "    {%- if message['role'] == 'system' -%}\n"
                      "        {%- if message['content'] -%}\n"
                      "            {{- message['content'] + '\\n\\n' -}}\n"
                      '        {%- endif -%}\n'
                      '        {%- if user_bio -%}\n'
                      "            {{- user_bio + '\\n\\n' -}}\n"
                      '        {%- endif -%}\n'
                      '    {%- else -%}\n'
                      "        {%- if message['role'] == 'user' -%}\n"
                      " 

# API extension 설치

In [19]:
!pip install -r ./extensions/api/requirements.txt
!pip install -r ./extensions/openai/requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: './extensions/api/requirements.txt'
ERROR: Could not open requirements file: [Errno 2] No such file or directory: './extensions/openai/requirements.txt'


# Server 구동

In [20]:
n_gpu_layers = 35
n_ctx = 5120

In [25]:
import llama_cpp
import os

print(os.path.exists("./models/Bookworm-10.7B-v0.4-DPO_q8_0.bin"))

llm = llama_cpp.Llama(model_path = "./models/Bookworm-10.7B-v0.4-DPO_q8_0.bin", n_gqa=8)

llama_model_load: error loading model: llama_model_loader: failed to load model from ./models/Bookworm-10.7B-v0.4-DPO_q8_0.bin

llama_load_model_from_file: failed to load model


True


ValueError: Failed to load model from file: ./models/Bookworm-10.7B-v0.4-DPO_q8_0.bin

In [21]:
!python server.py --model {model_file_name} --loader llamacpp --n-gpu-layers {str(n_gpu_layers)} --n_ctx {str(n_ctx)} \
 --public-api --share

06:54:50-468924 INFO     Starting Text generation web UI                                            
06:54:50-481657 INFO     Loading "Bookworm-10.7B-v0.4-DPO_q8_0.bin"                                 
06:54:50-722255 INFO     llama.cpp weights detected: "models/Bookworm-10.7B-v0.4-DPO_q8_0.bin"      
ggml_init_cublas: GGML_CUDA_FORCE_MMQ:   yes
ggml_init_cublas: CUDA_USE_TENSOR_CORES: no
ggml_init_cublas: found 1 CUDA devices:
  Device 0: Tesla T4, compute capability 7.5, VMM: yes
gguf_init_from_file: invalid magic characters '    '
llama_model_load: error loading model: llama_model_loader: failed to load model from models/Bookworm-10.7B-v0.4-DPO_q8_0.bin

llama_load_model_from_file: failed to load model
╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /content/text-generation-webui/server.py:241 in <module>                                         │
│                                                                                  

# Colab-SSH 설치 및 실행

In [12]:
# !pip install colab-ssh

In [13]:
# from colab_ssh import launch_ssh
# launch_ssh(NGROK_TOKEN, PASSWORD)

In [14]:
# import numpy as np

# I = np.eye(3)

# while True:
#   I = I@I